In [1]:
# # 🔹 Cell 1 — imports

import torch
import sys
sys.path.append("..")

from src.schema_parser import SchemaParser
from src.nl_parser import NLParser
from src.semantic_aligner import SemanticAligner
from src.schema_binder import bind_schema_tokens
from src.ast_renderer import SQLRenderer
from src.phase2_inference import infer_phase2_sql
from src.utils import (
    tokens_to_ids,
    ids_to_tokens,
    create_attention_mask,
    get_device
)

from src.vocab import PAD
from models.sql_transformer import SQLTransformer

In [2]:
# 🔹 Cell 2 — Load Phase-3 Model
device = get_device()

model = SQLTransformer().to(device)
# model.load_state_dict(
#     torch.load("checkpoints/phase3_model.pt", map_location=device)
# )
# model.eval()

# ✅ PHASE-3 LOADING SURGERY
PHASE3_CKPT = "checkpoints/phase3_model.pt"
checkpoint_state = torch.load(PHASE3_CKPT, map_location=device)
model_state = model.state_dict()

# Map old weights to new model indices (0-47)
mismatched_layers = ["embedding.weight", "fc_out.weight", "fc_out.bias"]
for name, param in checkpoint_state.items():
    if name in mismatched_layers:
        if len(param.shape) > 1:
            model_state[name][:param.shape[0], :] = param
        else:
            model_state[name][:param.shape[0]] = param
    else:
        model_state[name] = param

model.load_state_dict(model_state)
model.eval()
print("✅ Phase-3 nmodel loaded with Surgery (Size 49)")

✅ Phase-3 nmodel loaded with Surgery (Size 49)


In [12]:
# 🔹 Cell 3 — USER INPUT
USER_SCHEMA = {
    "schema_id": "U1",
    "tables": {
        "subject": ["id", "name", "marks"],
        "bat": ["id", "size", "weight"],
        "customers": ["id", "last_name", "age"],
        "orders": ["order_id", "customer_id", "order_date", "total_amount"],
        "products": ["product_id", "product_name", "category", "unit_price"],
        "employees": ["emp_id", "first_name", "department", "location", "salary"],
        "inventory": ["stock_id", "product_id", "warehouse_location", "quantity"],
        "departments": ["dept_id", "dept_name", "manager_id"],
        "suppliers": ["supplier_id", "company_name", "contact_email", "country"],
        "shipments": ["ship_id", "order_id", "tracking_number", "status"]
    }
}

#show average salary from employees
#get max salary from employees
#get average total_amount from orders
#show average salary from employees by department having average salary > 50000
#get max salary from employees by department having max salary > 80000
# get supplier_id from suppliers
# show average salary from employees by department and location
# get max salary from employees by department having max salary > 80000 and max salary < 150000
# get id from bat where size > 2
# show count of customers from customers by age having count greater than 10
# show sum total amount from orders by customer_id having sum greater than 100000 and sum less than 500000
# get ship_id, order_id from shipments
USER_NL_QUERY =  "show average salary from employees by department and location"


In [13]:
# # # 🔹 Cell 4 — inference
# def infer_phase3_sql(schema_json, nl_query):
#     # ==================================================
#     # 1️⃣ Schema parsing
#     # ==================================================
#     schema_parser = SchemaParser(schema_json)
#     tables = schema_parser.get_tables()
#     all_columns = schema_parser.get_all_columns()
#     #print(tables)
#     #print(all_columns)

#     # ==================================================
#     # 2️⃣ NL parsing
#     # ==================================================
#     nl_parser = NLParser()
#     signals = nl_parser.parse(nl_query)
#     nl_lower = nl_query.lower()
#     #print(signals)
#     # ==================================================
#     # 🔒 VALIDATION: HAVING requires aggregation
#     # ==================================================
#     if signals.get("having") and not signals["aggregations"]:
#         raise ValueError("❌ HAVING clause requires aggregation")

#     # ==================================================
#     # 3️⃣ Resolve TABLE
#     # ==================================================
#     resolved_table = None
#     for t in signals["entities"]:
#         if t in tables:
#             resolved_table = t
#             break

#     if resolved_table is None:
#         raise ValueError("❌ Could not resolve table from NL query")

#     # ==================================================
#     # 4️⃣ Semantic alignment
#     # ==================================================
#     aligner = SemanticAligner()
#     mapping = aligner.align(
#         user_terms=signals["entities"],
#         schema_terms=all_columns,
#         column_terms=all_columns
#     )
#     #print(mapping)
#     # ==================================================
#     # 5️⃣ Resolve SELECT (BACKWARD COMPATIBLE)
#     # ==================================================
#     select_items = []

#     is_pure_projection = (
#         not signals["aggregations"]
#         and not signals.get("group_by")
#         and not signals.get("having")
#     )

#     # ----------------------------------
#     # 🔁 Phase-1 / Phase-2 behavior
#     # ----------------------------------
#     if is_pure_projection:
#         for term, mapped in mapping.items():
#             if mapped.startswith(resolved_table + ".") and term in nl_lower:
#                 select_items.append({
#                     "agg": None,
#                     "column": mapped.split(".")[1]
#                 })

#         if not select_items:
#             raise ValueError("❌ No SELECT columns resolved")

#     # ----------------------------------
#     # 🧠 Phase-3 aggregation behavior
#     # ----------------------------------
#     else:
#         agg = signals["aggregations"][0].upper()

#         preferred_cols = [
#             mapped for term, mapped in mapping.items()
#             if mapped.startswith(resolved_table + ".") and term in nl_lower
#         ]

#         if not preferred_cols:
#             heuristic_numeric = {
#                 "salary", "age", "marks", "amount",
#                 "price", "quantity", "weight", "total"
#             }
#             preferred_cols = [
#                 f"{resolved_table}.{c}"
#                 for c in schema_parser.get_columns(resolved_table)
#                 if c in heuristic_numeric
#             ]

#         if not preferred_cols:
#             raise ValueError("❌ No valid aggregation column found")

#         select_items.append({
#             "agg": agg,
#             "column": preferred_cols[0].split(".")[1]
#         })

#     # ==================================================
#     # 6️⃣ GROUP BY (MULTI + SAFE)
#     # ==================================================
#     group_by_cols = []
#     seen = set()

#     agg_col = select_items[0]["column"] if select_items[0]["agg"] else None

#     for g in signals.get("group_by", []):
#         if g == agg_col:
#             continue

#         col = schema_parser.resolve_column(resolved_table, g)
#         if col and col not in seen:
#             group_by_cols.append(col)
#             seen.add(col)

#     # ==================================================
#     # 7️⃣ HAVING (MULTI + BACKWARD SAFE)
#     # ==================================================
#     having_clause = []

#     # ----------------------------------
#     # ✅ MULTI-HAVING (preferred)
#     # ----------------------------------
#     if signals.get("having_conditions"):
#         for cond in signals["having_conditions"]:
#             having_clause.append({
#                 "agg": cond["agg"].upper(),
#                 "column": select_items[0]["column"],
#                 "op": cond["op"],
#                 "value": cond["value"]
#             })

#     # ----------------------------------
#     # 🔁 BACKWARD FALLBACK (single HAVING)
#     # ----------------------------------
#     elif signals.get("having"):
#         having_agg = (
#             signals["having_agg"].upper()
#             if signals.get("having_agg")
#             else signals["aggregations"][0].upper()
#         )

#         having_clause.append({
#             "agg": having_agg,
#             "column": select_items[0]["column"],
#             "op": signals["operator"],
#             "value": signals["having_value"]
#         })

#     # ==================================================
#     # 🔟 Render SQL
#     # ==================================================
#     renderer = SQLRenderer()
#     return renderer.render({
#         "select": select_items,
#         "from": [resolved_table],
#         "where": [],
#         "group_by": group_by_cols,
#         "having": having_clause,
#         "having_logic": signals.get("having_logic", "AND")
#     })
#     return sql

from src.phase3_inference import infer_phase3_sql

In [14]:
# 🔹 Cell 4.5 — Query Router (Phase Selector)

def route_and_infer(schema_json, nl_query):
    signals = NLParser().parse(nl_query)

    # 🔀 Phase selection (STRICT & SAFE)
    if (
        signals["aggregations"]
        or signals.get("group_by")
        or signals.get("having")
    ):
        return infer_phase3_sql(schema_json, nl_query)
    else:
        return infer_phase2_sql(schema_json, nl_query)

In [15]:
# Cell 5 — Run Phase-3 Inference
sql_output = route_and_infer(USER_SCHEMA, USER_NL_QUERY)

print("🧠 NL Query :", USER_NL_QUERY)
print("🧾 SQL Query:", sql_output)

Low confidence for 'average', skipped


🧠 NL Query : show average salary from employees by department and location
🧾 SQL Query: SELECT AVG(salary) FROM employees GROUP BY employees.department, employees.location
